In [486]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import time
%matplotlib inline


gameid = 2022090800
playid = 56
frameid = 159

routes_df = pd.read_csv('../route_frames.csv')
routes_df['dir_clean'] = routes_df['dir'] * -np.pi / 180 + np.pi/2

coverage_df = pd.read_csv('../coverage_frames.csv')
coverage_df['dir_clean'] = coverage_df['dir'] * -np.pi / 180 + np.pi/2

ball_df = pd.read_csv('../ball_locations.csv')
ball_df = ball_df[['gameId', 'playId', 'frameId', 'x', 'y']]

stacked_df = pd.concat([routes_df, coverage_df])

stacked_df = stacked_df.merge(ball_df, how='left', on=['gameId', 'playId', 'frameId'], suffixes=('', '_ball'))


stacked_df['dis_to_ball'] = np.sqrt((stacked_df.x - stacked_df.x_ball)**2 + (stacked_df.y - stacked_df.y_ball)**2)

testframe = stacked_df[(stacked_df['gameId'] == gameid) & (stacked_df['playId'] == playid) & (stacked_df['frameId'] == frameid)]
testplay = stacked_df[(stacked_df['gameId'] == gameid) & (stacked_df['playId'] == playid)]

route_runners = pd.read_csv('../route_runners.csv')

In [ ]:
x = np.linspace(0, 50, 100)

plt.ylim(0, 15)
def func(x):
    return np.minimum(np.repeat(8, len(x)), 2 + np.exp(x/16))

plt.plot(x, func(x))

In [ ]:
def get_mean(frame: pd.DataFrame) -> np.ndarray:

    mu_array = np.stack([
        (.5 *
        frame['s'] *
        np.cos(frame['dir_clean'])) + frame['x'],
        (.5 *
        frame['s'] *
        np.sin(frame['dir_clean'])) + frame['y']
    ], axis=1)
    
    return mu_array


def get_cov_matricies(frame: pd.DataFrame) -> np.ndarray:    
    
    # player influence hyperparameters
    #scaling_fn = lambda x: np.repeat(4, len(x)) # influence radius relation with distance to ball
    scaling_fn = lambda x: np.minimum(np.repeat(8, len(x)), 2 + np.exp(x/16))

    speed_ratios = (frame['s'].to_numpy() ** 2) / (18 ** 2)

    distances = frame['dis_to_ball'].to_numpy()
    diagonals = np.stack([
                    np.array(
                        (scaling_fn(distances) + scaling_fn(distances) * speed_ratios) / 2
                    ),
                    np.array(
                        (scaling_fn(distances) - scaling_fn(distances) * speed_ratios) / 2
                    )]
                , axis=1)
        
    scaling_matricies = diagonals[:, None, :] * np.eye(2)

    thetas = frame['dir_clean'].to_numpy()
    rotation_matricies = np.stack([
        np.stack([np.cos(thetas), -np.sin(thetas)], axis=1),
        np.stack([np.sin(thetas), np.cos(thetas)], axis=1)
    ], axis=1)

    covariance_matricies = rotation_matricies @ scaling_matricies @ scaling_matricies @ np.transpose(rotation_matricies, (0,2,1))

    return covariance_matricies


def get_bboxes(means: np.ndarray, covariances: np.ndarray) -> np.ndarray:
    eigvals, eigvecs = np.linalg.eigh(covariances)

    major_axes = np.sqrt(eigvals[:, 1])
    minor_axes = np.sqrt(eigvals[:, 0])

    thetas = np.arctan2(eigvecs[:, 1, 1], eigvecs[:, 0, 1])

    t = np.tile(np.linspace(0, 2*np.pi, 100), (means.shape[0], 1))

    ellipses_x = np.reshape(major_axes, (means.shape[0],1)) * np.cos(t) 
    ellipses_y = np.reshape(minor_axes, (means.shape[0],1)) * np.sin(t) 
    
    rotation_matricies = np.stack([
        np.stack([np.cos(thetas), -np.sin(thetas)], axis=1),
        np.stack([np.sin(thetas), np.cos(thetas)], axis=1)
    ], axis=1)
    
    rotated_ellipses = rotation_matricies @ np.stack([ellipses_x, ellipses_y], axis=1)


    ellipse_x_final = rotated_ellipses[:, 0, :] + np.reshape(means[:, 0], (means.shape[0], 1))
    ellipse_y_final = rotated_ellipses[:, 1, :] + np.reshape(means[:, 1], (means.shape[0], 1))

    bboxes = np.stack([
        np.min(ellipse_x_final, axis=1), 
        np.max(ellipse_x_final, axis=1), 
        np.min(ellipse_y_final, axis=1), 
        np.max(ellipse_y_final, axis=1)
    ], axis=1)

    return bboxes

    
def filter_defender_bboxes(offense: np.ndarray, defense: np.ndarray):
    
    # cartesian product of offense and defense bboxes
    offense_repeated = np.repeat(offense, defense.shape[0], axis=0)
    defense_repeated = np.tile(defense, (offense.shape[0], 1))

    arr = ~(
        (offense_repeated[:,1] < defense_repeated[:,0]) |
        (defense_repeated[:,1] < offense_repeated[:,0]) |
        (offense_repeated[:,3] < defense_repeated[:,2]) |
        (defense_repeated[:,3] < offense_repeated[:,2])
    )
    return arr.reshape((offense.shape[0], defense.shape[0]))


def check_points_in_offense_ellipses(x: np.ndarray, y: np.ndarray, means: np.ndarray, covariances: np.ndarray):

    # would like to have this function work for if means and covariances have an extra dimension
    
    eigvals, eigvecs = np.linalg.eigh(covariances)

    points = np.stack([x, y], axis=1) - means # shift by mean

    major_axes_len = np.sqrt(eigvals[:, 1])
    minor_axes_len = np.sqrt(eigvals[:, 0])

    major_axes_dir = eigvecs[:, :, 1]
    minor_axes_dir = eigvecs[:, :, 0]
    
    transformed_points = np.stack([
            np.diag(points @ major_axes_dir.T) / major_axes_len,
            np.diag(points @ minor_axes_dir.T) / minor_axes_len
        ], axis=1)
    
    return np.sum(transformed_points ** 2, axis=1) <= 1


def check_points_in_defense_ellipses(x: np.ndarray, y: np.ndarray, means: np.ndarray, covariances: np.ndarray, filter_array: np.ndarray):
    
    # need to check each poitn on each defender
    eigvals, eigvecs = np.linalg.eigh(covariances)

    # shift each point by each mean
    points = np.stack([x, y], axis=1)
    filtered_means = (
        np.tile(
            means, (points.shape[0], 1, 1)
        ).T * filter_array.T
    ).T

    expanded_points = np.transpose(
        np.tile(
            points, (means.shape[0], 1, 1)
        ), (1, 0, 2)
    )

    filtered_points = (
        expanded_points.T * filter_array.T
    ).T
    
    shifted_points = filtered_points - filtered_means

    major_axes_len = np.sqrt(eigvals[:, 1])
    minor_axes_len = np.sqrt(eigvals[:, 0])

    major_axes_dir = eigvecs[:, :, 1]
    minor_axes_dir = eigvecs[:, :, 0]

    filtered_major_evecs = (
        np.transpose(
            np.tile(
                major_axes_dir, (points.shape[0], 1, 1)
            )
        ,(2, 1, 0)) * filter_array.T
    ).T

    filtered_minor_evecs = (
        np.transpose(
            np.tile(
                minor_axes_dir, (points.shape[0], 1, 1)
            )
        ,(2, 1, 0)) * filter_array.T
    ).T
    
    transformed_points = np.stack([
        np.diagonal(
            shifted_points @ np.transpose(
                filtered_major_evecs, (0, 2, 1)
            ), axis1=1, axis2=2
        ) / major_axes_len,
        np.diagonal(
            shifted_points @ np.transpose(
                filtered_minor_evecs, (0, 2, 1)
            ), axis1=1, axis2=2
        ) / minor_axes_len,
    ], axis=1)

    return np.any((np.sum(transformed_points ** 2, axis=1) <= 1) & filter_array, axis=1)


def get_bbox_areas(bboxes):
    return (bboxes[:, 1] - bboxes[:, 0]) * (bboxes[:, 3] - bboxes[:, 2]) # xmax - xmin * ymax - ymin



def get_openness(frame: pd.DataFrame):
    offense = frame[frame['wasRunningRoute'] == 1].copy()
    defense = frame[frame['wasRunningRoute'].isna()].copy()

    if offense.shape[0] == 0:
        return False

    # mean and covariance matricies for offense
    offense_means = get_mean(offense) # n, 2, 1
    offense_covs = get_cov_matricies(offense) # n, 2, 2

    # mean and covariance matricies for defense
    defense_means = get_mean(defense) 
    defense_covs = get_cov_matricies(defense)

    offense_bboxes = get_bboxes(offense_means, offense_covs)
    defense_bboxes = get_bboxes(defense_means, defense_covs)

    offense_bbox_areas = get_bbox_areas(offense_bboxes)

    # create a search grid for each offense bbox
    n = 50
    search_spaces = np.array([np.meshgrid(np.linspace(xmin, xmax, n), np.linspace(ymin, ymax, n)) for xmin, xmax, ymin, ymax in offense_bboxes])


    filter_arr = filter_defender_bboxes(offense_bboxes, defense_bboxes)


    # create counts array for each offensive player initialized at zero
    counts = np.zeros(offense.shape[0])

    # iterate over indecies for search spaces
    for i in range(n):
        for j in range(n):
            
            x_points = search_spaces[:, 0, i, j]
            y_points = search_spaces[:, 1, i, j]

            in_offense_ellipses = check_points_in_offense_ellipses(
                x_points, y_points, offense_means, offense_covs
            )

            in_any_defense_ellipses = check_points_in_defense_ellipses(
                x_points, y_points, defense_means, defense_covs, filter_arr
            )
            

            counts += (in_offense_ellipses) & (in_any_defense_ellipses)

    intersection_areas = offense_bbox_areas * (counts / n ** 2)

    return 1 - (intersection_areas / offense_bbox_areas)


get_openness(testframe)

In [ ]:
def plot_frame(play_df: pd.DataFrame, frame):
    frame_df = play_df[play_df['frameId'] == frame]
    
    fig, ax = plt.subplots(figsize=(6,6))
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title('Player Influence Zones')
    ax.set_ylim(0, 53.3)
    ax.set_xlim(frame_df['x'].min() - 10, frame_df['x'].max() + 10)

    means = get_mean(frame_df)
    covs = get_cov_matricies(frame_df)
    
    eigvals, eigvecs = np.linalg.eigh(covs)

    major_axes = np.sqrt(eigvals[:, 1])
    minor_axes = np.sqrt(eigvals[:, 0])

    thetas = np.arctan2(eigvecs[:, 1, 1], eigvecs[:, 0, 1])

    t = np.tile(np.linspace(0, 2*np.pi, 100), (means.shape[0], 1))

    ellipses_x = np.reshape(major_axes, (means.shape[0], 1)) * np.cos(t) 
    ellipses_y = np.reshape(minor_axes, (means.shape[0], 1)) * np.sin(t) 
    
    rotation_matricies = np.stack([
        np.stack([np.cos(thetas), -np.sin(thetas)], axis=1),
        np.stack([np.sin(thetas), np.cos(thetas)], axis=1)
    ], axis=1)
    
    rotated_ellipses = rotation_matricies @ np.stack([ellipses_x, ellipses_y], axis=1)


    ellipse_x_final = rotated_ellipses[:, 0, :] + np.reshape(means[:, 0], (means.shape[0], 1))
    ellipse_y_final = rotated_ellipses[:, 1, :] + np.reshape(means[:, 1], (means.shape[0], 1))

    colors = ['blue', 'red']


    for i, (ellipse_x, ellipse_y) in enumerate(zip(ellipse_x_final, ellipse_y_final)):
        ax.plot(ellipse_x, ellipse_y, color=colors[pd.isna(frame_df.iloc[i].wasRunningRoute)])
        ax.scatter(frame_df.iloc[i].x, frame_df.iloc[i].y, color=colors[pd.isna(frame_df.iloc[i].wasRunningRoute)], marker='x')
        ax.arrow(frame_df.iloc[i].x, frame_df.iloc[i].y, means[i, 0] - frame_df.iloc[i].x, means[i, 1] - frame_df.iloc[i].y, head_width = .5, head_length=.2, color='black')


    return 

frameids = sorted(testplay.frameId.unique())


plot_frame(testplay, frameid)

In [473]:
temp = (
    testplay
    .merge(route_frame_cutoffs.reset_index(), how='left', on='routeRan')
    .merge(start_frames, how='left', on=['gameId', 'playId'])
)

temp['valid'] = (temp['frameId'] - temp['start_frame'] >= temp['low_perc']) & (temp['frameId'] - temp['start_frame'] <= temp['high_perc'])

In [ ]:
stacked_df.playId.unique()

In [ ]:
playid = 122
testplay = stacked_df[(stacked_df['gameId'] == gameid) & (stacked_df['playId'] == playid)]
testplay

In [ ]:

def animate_play(play_df: pd.DataFrame):
    frameid_list = sorted(play_df['frameId'].unique())

    fig, axes = plt.subplots(nrows=2, figsize=(6,8))
    axes[0].set_xlabel('X')
    axes[0].set_ylabel('Y')
    axes[0].set_title('Player Influence Zones')
    axes[0].set_ylim(0, 53.3)
    axes[0].set_xlim()

    axes[1].set_xlabel('Frame')
    axes[1].set_ylabel('Openness')
    axes[1].set_ylim((.5, 1.01))

    first_frame = play_df[play_df['frameId'] == frameid_list[0]]

    colors = ['blue', 'red']
    open_arrs = {row.nflId: [] for _, row in first_frame[first_frame['wasRunningRoute'] == 1].iterrows()}

    def update(frame):
        frame_df = play_df[play_df['frameId'] == frameid_list[frame]]

        axes[0].clear()
        axes[0].set_xlim(frame_df['x'].min() - 10, frame_df['x'].max() + 10)

        means = get_mean(frame_df)
        covs = get_cov_matricies(frame_df)
        
        eigvals, eigvecs = np.linalg.eigh(covs)

        major_axes = np.sqrt(eigvals[:, 1])
        minor_axes = np.sqrt(eigvals[:, 0])

        thetas = np.arctan2(eigvecs[:, 1, 1], eigvecs[:, 0, 1])

        t = np.tile(np.linspace(0, 2*np.pi, 100), (means.shape[0], 1))

        ellipses_x = np.reshape(major_axes, (means.shape[0], 1)) * np.cos(t) 
        ellipses_y = np.reshape(minor_axes, (means.shape[0], 1)) * np.sin(t) 
        
        rotation_matricies = np.stack([
            np.stack([np.cos(thetas), -np.sin(thetas)], axis=1),
            np.stack([np.sin(thetas), np.cos(thetas)], axis=1)
        ], axis=1)
        
        rotated_ellipses = rotation_matricies @ np.stack([ellipses_x, ellipses_y], axis=1)


        ellipse_x_final = rotated_ellipses[:, 0, :] + np.reshape(means[:, 0], (means.shape[0], 1))
        ellipse_y_final = rotated_ellipses[:, 1, :] + np.reshape(means[:, 1], (means.shape[0], 1))



        for i, (ellipse_x, ellipse_y) in enumerate(zip(ellipse_x_final, ellipse_y_final)):
            axes[0].plot(ellipse_x, ellipse_y, color=colors[pd.isna(frame_df.iloc[i].wasRunningRoute)])
            axes[0].scatter(frame_df.iloc[i].x, frame_df.iloc[i].y, color=colors[pd.isna(frame_df.iloc[i].wasRunningRoute)], marker='x')
            axes[0].arrow(frame_df.iloc[i].x, frame_df.iloc[i].y, means[i, 0] - frame_df.iloc[i].x, means[i, 1] - frame_df.iloc[i].y, head_width = .5, head_length=.2, color='black')

        open_arr = get_openness(frame_df)
        print(open_arr)

        offense = frame_df.loc[frame_df['wasRunningRoute'] == 1].copy()

        if not np.array(open_arr).any():
            return
        
        for id, open in open_arr:
            open_arrs[id] += [open]

            newcolors = ['grey', 'blue']
            
            axes[1].plot(range(len(open_arrs[id])), open_arrs[id], color='blue')

        

        return 
    
    anim = animation.FuncAnimation(fig, update, frames=len(frameid_list), interval=150)
    anim.save('anim.gif')
    return 


animate_play(testplay)


In [ ]:
from IPython.display import display, Image

# Display a GIF file
display(Image(filename="anim.gif"))

In [501]:
def get_mean(frame: pd.DataFrame) -> np.ndarray:

    mu_array = np.stack([
        (.5 *
        frame['s'] *
        np.cos(frame['dir_clean'])) + frame['x'],
        (.5 *
        frame['s'] *
        np.sin(frame['dir_clean'])) + frame['y']
    ], axis=1)
    
    return mu_array


def get_cov_matricies(frame: pd.DataFrame) -> np.ndarray:    
    
    # player influence hyperparameters
    scaling_fn = lambda x: np.minimum(np.repeat(8, len(x)), 2 + np.exp(x/16)) # influence radius relation with distance to ball

    speed_ratios = (frame['s'].to_numpy() ** 2) / (18 ** 2)

    distances = frame['dis_to_ball'].to_numpy()
    diagonals = np.stack([
                    np.array(
                        (scaling_fn(distances) + scaling_fn(distances) * speed_ratios) / 2
                    ),
                    np.array(
                        (scaling_fn(distances) - scaling_fn(distances) * speed_ratios) / 2
                    )]
                , axis=1)
        
    scaling_matricies = diagonals[:, None, :] * np.eye(2)

    thetas = frame['dir_clean'].to_numpy()
    rotation_matricies = np.stack([
        np.stack([np.cos(thetas), -np.sin(thetas)], axis=1),
        np.stack([np.sin(thetas), np.cos(thetas)], axis=1)
    ], axis=1)

    covariance_matricies = rotation_matricies @ scaling_matricies @ scaling_matricies @ np.transpose(rotation_matricies, (0,2,1))

    return covariance_matricies


def get_bboxes(means: np.ndarray, covariances: np.ndarray) -> np.ndarray:
    eigvals, eigvecs = np.linalg.eigh(covariances)

    major_axes = np.sqrt(eigvals[:, 1])
    minor_axes = np.sqrt(eigvals[:, 0])

    thetas = np.arctan2(eigvecs[:, 1, 1], eigvecs[:, 0, 1])

    t = np.tile(np.linspace(0, 2*np.pi, 100), (means.shape[0], 1))

    ellipses_x = np.reshape(major_axes, (means.shape[0],1)) * np.cos(t) 
    ellipses_y = np.reshape(minor_axes, (means.shape[0],1)) * np.sin(t) 
    
    rotation_matricies = np.stack([
        np.stack([np.cos(thetas), -np.sin(thetas)], axis=1),
        np.stack([np.sin(thetas), np.cos(thetas)], axis=1)
    ], axis=1)
    
    rotated_ellipses = rotation_matricies @ np.stack([ellipses_x, ellipses_y], axis=1)


    ellipse_x_final = rotated_ellipses[:, 0, :] + np.reshape(means[:, 0], (means.shape[0], 1))
    ellipse_y_final = rotated_ellipses[:, 1, :] + np.reshape(means[:, 1], (means.shape[0], 1))

    bboxes = np.stack([
        np.min(ellipse_x_final, axis=1), 
        np.max(ellipse_x_final, axis=1), 
        np.min(ellipse_y_final, axis=1), 
        np.max(ellipse_y_final, axis=1)
    ], axis=1)

    return bboxes

    
def filter_defender_bboxes(offense: np.ndarray, defense: np.ndarray):
    
    # cartesian product of offense and defense bboxes
    offense_repeated = np.repeat(offense, defense.shape[0], axis=0)
    defense_repeated = np.tile(defense, (offense.shape[0], 1))

    arr = ~(
        (offense_repeated[:,1] < defense_repeated[:,0]) |
        (defense_repeated[:,1] < offense_repeated[:,0]) |
        (offense_repeated[:,3] < defense_repeated[:,2]) |
        (defense_repeated[:,3] < offense_repeated[:,2])
    )
    return arr.reshape((offense.shape[0], defense.shape[0]))


def check_points_in_offense_ellipses(x: np.ndarray, y: np.ndarray, means: np.ndarray, covariances: np.ndarray):

    # would like to have this function work for if means and covariances have an extra dimension
    
    eigvals, eigvecs = np.linalg.eigh(covariances)

    points = np.stack([x, y], axis=1) - means # shift by mean

    major_axes_len = np.sqrt(eigvals[:, 1])
    minor_axes_len = np.sqrt(eigvals[:, 0])

    major_axes_dir = eigvecs[:, :, 1]
    minor_axes_dir = eigvecs[:, :, 0]
    
    transformed_points = np.stack([
            np.diag(points @ major_axes_dir.T) / major_axes_len,
            np.diag(points @ minor_axes_dir.T) / minor_axes_len
        ], axis=1)
    
    return np.sum(transformed_points ** 2, axis=1) <= 1


def check_points_in_defense_ellipses(x: np.ndarray, y: np.ndarray, means: np.ndarray, covariances: np.ndarray, filter_array: np.ndarray):
    
    # need to check each poitn on each defender
    eigvals, eigvecs = np.linalg.eigh(covariances)

    # shift each point by each mean
    points = np.stack([x, y], axis=1)
    filtered_means = (
        np.tile(
            means, (points.shape[0], 1, 1)
        ).T * filter_array.T
    ).T

    expanded_points = np.transpose(
        np.tile(
            points, (means.shape[0], 1, 1)
        ), (1, 0, 2)
    )

    filtered_points = (
        expanded_points.T * filter_array.T
    ).T
    
    shifted_points = filtered_points - filtered_means

    major_axes_len = np.sqrt(eigvals[:, 1])
    minor_axes_len = np.sqrt(eigvals[:, 0])

    major_axes_dir = eigvecs[:, :, 1]
    minor_axes_dir = eigvecs[:, :, 0]

    filtered_major_evecs = (
        np.transpose(
            np.tile(
                major_axes_dir, (points.shape[0], 1, 1)
            )
        ,(2, 1, 0)) * filter_array.T
    ).T

    filtered_minor_evecs = (
        np.transpose(
            np.tile(
                minor_axes_dir, (points.shape[0], 1, 1)
            )
        ,(2, 1, 0)) * filter_array.T
    ).T
    
    transformed_points = np.stack([
        np.diagonal(
            shifted_points @ np.transpose(
                filtered_major_evecs, (0, 2, 1)
            ), axis1=1, axis2=2
        ) / major_axes_len,
        np.diagonal(
            shifted_points @ np.transpose(
                filtered_minor_evecs, (0, 2, 1)
            ), axis1=1, axis2=2
        ) / minor_axes_len,
    ], axis=1)

    return np.any((np.sum(transformed_points ** 2, axis=1) <= 1) & filter_array, axis=1)


def get_bbox_areas(bboxes):
    return (bboxes[:, 1] - bboxes[:, 0]) * (bboxes[:, 3] - bboxes[:, 2]) # xmax - xmin * ymax - ymin



def get_openness(frame: pd.DataFrame):
    offense = frame[frame['wasRunningRoute'] == 1] # wasRunningRoute == 1
    defense = frame[frame['wasRunningRoute'] != 1] # wasRunningRoute == NA

    if offense.shape[0] == 0 or offense['frame_clean'].max() < 5:
        return []
        # return nas if no offense

    # get offense nflIds
    offense_ids = offense['nflId']


    # mean and covariance matricies for offense
    offense_means = get_mean(offense) # n, 2, 1
    offense_covs = get_cov_matricies(offense) # n, 2, 2

    # mean and covariance matricies for defense
    defense_means = get_mean(defense) 
    defense_covs = get_cov_matricies(defense)

    offense_bboxes = get_bboxes(offense_means, offense_covs)
    defense_bboxes = get_bboxes(defense_means, defense_covs)

    offense_bbox_areas = get_bbox_areas(offense_bboxes)

    # create a search grid for each offense bbox
    n = 50
    search_spaces = np.array([np.meshgrid(np.linspace(xmin, xmax, n), np.linspace(ymin, ymax, n)) for xmin, xmax, ymin, ymax in offense_bboxes])

    filter_arr = filter_defender_bboxes(offense_bboxes, defense_bboxes)


    # create counts array for each offensive player initialized at zero
    counts = np.zeros(offense.shape[0])

    # iterate over indecies for search spaces
    for i in range(n):
        for j in range(n):
            
            x_points = search_spaces[:, 0, i, j]
            y_points = search_spaces[:, 1, i, j]

            in_offense_ellipses = check_points_in_offense_ellipses(
                x_points, y_points, offense_means, offense_covs
            )

            in_any_defense_ellipses = check_points_in_defense_ellipses(
                x_points, y_points, defense_means, defense_covs, filter_arr
            )
            

            counts += (in_offense_ellipses) & (in_any_defense_ellipses)

    intersection_areas = offense_bbox_areas * (counts / n ** 2)

    offense_opennesses = 1 - (intersection_areas / offense_bbox_areas)
    return list(zip(offense_ids, offense_opennesses))

    offense_bools = frame['wasRunningRoute'] == 1
    result = np.repeat(np.NaN, frame.shape[0])
    result[offense_bools] = offense_opennesses
    
    return result

#get_openness(testframe)

In [495]:
testplay['frame_clean'] = (
    testplay
    .loc[testplay['wasTargettedReceiver'] == 1]
    .groupby(['gameId', 'playId', 'nflId'])
    ['frameId']
    .transform(lambda x: x - x.min())
)

In [ ]:
stacked_df['frame_clean'] = (
    stacked_df
    .loc[stacked_df['wasTargettedReceiver'] == 1]
    .groupby(['gameId', 'playId', 'nflId'])
    ['frameId']
    .transform(lambda x: x - x.min())
)

targetted_df = stacked_df.loc[stacked_df['wasTargettedReceiver'] == 1].copy()


targetted_df['had_forward_pass'] = (
    targetted_df
    .groupby(['gameId', 'playId', 'nflId'])
    ['event']
    .transform(lambda x: 'pass_forward' in x.to_list())
)

def low_perc(x):
    return np.percentile(x, 10)

def high_perc(x):
    return np.percentile(x, 90)

route_frame_cutoffs = (
    targetted_df.loc[(targetted_df['had_forward_pass']) & (targetted_df['event'] == 'pass_forward')]
    .groupby('routeRan')
    ['frame_clean']
    .agg([low_perc, 'mean', high_perc])
    #.rename(columns={'low_perc': '5th Percentile', 'high_perc': '95th Percentile'})
)

route_frame_cutoffs.reset_index()
print(route_frame_cutoffs.reset_index().to_markdown())

In [ ]:
# should take about 9hr
result = (
    stacked_df
    .groupby(['gameId', 'playId', 'frameId'])
    #[col_list]
    .apply(get_openness)
)
open_df = result.explode().reset_index()
open_df['nflId'] = open_df[0].str[0]
open_df['openness'] = open_df[0].str[1]
open_df.drop(columns=0) # final time 69 minutes

In [ ]:
routes = stacked_df.groupby(['gameId', 'playId', 'nflId'])['routeRan'].first().reset_index()
route_frame_cutoffs.reset_index()

In [521]:
start_frames = (
    stacked_df
    .groupby(['gameId', 'playId'])
    ['frameId']
    .min()
    .reset_index()
    .rename(columns={'frameId': 'start_frame'})
)

In [522]:
open_df_with_routes = (
    open_df
    .loc[open_df['nflId'].notna()]
    .merge(routes, how='left', on=['gameId', 'playId', 'nflId'])
    .merge(route_frame_cutoffs.reset_index(), how='left', on='routeRan')
    .merge(start_frames, how='left', on=['gameId', 'playId'])
)

filtered_open_df = (
    open_df_with_routes
    .loc[
        ((open_df_with_routes['frameId'] - open_df_with_routes['start_frame']) >= open_df_with_routes['low_perc']) & 
        ((open_df_with_routes['frameId'] - open_df_with_routes['start_frame']) <= open_df_with_routes['high_perc'])
    ]
)

In [ ]:
play_aggregated_df = filtered_open_df.groupby(['gameId', 'playId', 'nflId'])['openness'].agg('mean').reset_index()
play_aggregated_df

In [ ]:
player_aggregated_df = play_aggregated_df.groupby('nflId')['openness'].agg(['mean', 'count'])
player_aggregated_df.reset_index().sort_values(['mean', 'count'], ascending=False)

In [537]:
play_aggregated_df.to_csv('9_week_play_aggregated.csv')
player_aggregated_df.to_csv('9_week_player_aggregated.csv')

In [ ]:
combine = pd.read_csv('../data/filtered_combine.csv')
play_aggregated_df.merge(combine, left_on='')

In [559]:
t = player_aggregated_df.loc[player_aggregated_df['count'] >= 200, ['mean', 'count']].reset_index().sort_values(['mean', 'count'], ascending=False).merge(pd.read_csv('../route_runners.csv'), how='left', on='nflId')

In [ ]:
(
    play_aggregated_df
    .merge(routes, how='left', on=['gameId', 'playId', 'nflId'])
    .groupby(['nflId', 'routeRan'])
    ['openness']
    .agg(['mean', 'count'])
    .reset_index()
    .sort_values(['mean', 'count'], ascending=False)
    .merge(route_runners, how='left', on='nflId')
)

In [ ]:
t[t['position'] == 'WR'].head(60)

In [ ]:
import matplotlib.pyplot as plt

def draw_nfl_field(ax):
    field_length = 40
    field_width = 20
    hash_dist = field_width / 2  # Hash marks closer to the center
    yard_line_spacing = 10  # Every 5 yards

    # Set background color with transparency
    ax.set_facecolor((0.0, 0.5, 0.0, 0.6))  # RGBA (Green with transparency)

    # Draw outer boundary
    ax.plot([0, 0, field_length, field_length, 0], 
            [0, field_width, field_width, 0, 0], 
            'black', linewidth=2)

    # Draw yard lines
    for yd in range(0, field_length + 1, yard_line_spacing):
        ax.plot([yd, yd], [0, field_width], 'gray', linestyle="--", linewidth=0.8)

    # Draw hash marks (closer to center)
    for yd in range(1, field_length):  # Exclude goal lines
        for y in [field_width / 2 - hash_dist / 2, field_width / 2 + hash_dist / 2]:  
            ax.plot([yd, yd], [y - 0.3, y + 0.3], 'black', linewidth=1)

    # Draw midfield 50-yard line
    #ax.plot([50, 50], [0, field_width], 'black', linewidth=2)

    # Remove axes
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlim(0, field_length)
    ax.set_ylim(0, field_width)
    ax.set_aspect(1)

# Create figure with two subplots
fig, axes = plt.subplots(1, 2, figsize=(8, 3))

# Draw two NFL fields
for ax in axes:
    draw_nfl_field(ax)


x = [18.5, 20]
y = [11, 10]
s = [12, 4]
dir = [155/180 * np.pi, 75/180 * np.pi]
ball = [35, 25]
teams = ['offense', 'defense']
colors = ['blue', 'red']

axes[0].scatter(x[0], y[0], color=colors[0], label=teams[0])
axes[0].scatter(x[1], y[1], color=colors[1], label=teams[1])
axes[0].set_title('Low Euclidean Distance')
axes[0].legend()


x = [20, 12]
y = [10, 10]
dir = [75/180 * np.pi, 155/180 * np.pi]
ball = [35, 25]

axes[1].scatter(x[0], y[0], color=colors[0], label=teams[0])
axes[1].scatter(x[1], y[1], color=colors[1], label=teams[1])
axes[1].set_title('High Euclidean Distance')
axes[1].legend()

plt.show()

In [466]:
import numpy as np
import matplotlib.pyplot as plt
def get_cov_matrix(s, d, theta):
    
    scaling_fn = lambda x: min(8, 2 + np.exp(x/16))
    speed_ratio = s**2 / 18**2 # speed ratio (18 is max speed)

    scaling_matrix = np.array([
        [(scaling_fn(d) + (scaling_fn(d) * speed_ratio)) / 2, 0],
        [0, (scaling_fn(d) - (scaling_fn(d) * speed_ratio)) / 2]    
    ])
    rotation_matrix = np.array([
        [np.cos(theta), -np.sin(theta)], 
        [np.sin(theta), np.cos(theta)]
    ])

    covariance_matrix = rotation_matrix @ scaling_matrix @ scaling_matrix @ np.linalg.inv(rotation_matrix)

    return covariance_matrix


def get_mean(xi, yi, s, theta):
    mu = np.array([xi, yi]) + np.array([s * np.cos(theta), s * np.sin(theta)]) * .5
    return mu

def plot_sample(x_, y_, s_, dir_, ball_, team_, color_, ax):
    for i in range(2):
        x = x_[i]
        y = y_[i]
        s = s_[i]
        
        
        team = team_[i]
        color = color_[i]
        dis_to_ball = np.sqrt((ball_[0] - x) ** 2 - (ball_[1] - y) ** 2)
        theta = dir_[i]

        # Given mean and covariance
        mu = get_mean(x, y, s, theta)# Mean vector
        Sigma = get_cov_matrix(s, dis_to_ball, theta)  # Covariance matrix

        # Given PDF value
        for n in [.1, .2, .4, .8, 1.6]:

            # Compute determinant and inverse of Sigma
            det_Sigma = np.linalg.det(Sigma)

            # Eigen decomposition of covariance matrix
            eigvals, eigvecs = np.linalg.eigh(Sigma)

            # Compute semi-axis lengths
            a = n * np.sqrt(eigvals[1])  # Major axis
            b = n * np.sqrt(eigvals[0])  # Minor axis

            # Compute rotation angle
            theta = np.arctan2(eigvecs[1, 1], eigvecs[0, 1])

            # Generate ellipse points
            t = np.linspace(0, 2 * np.pi, 100)
            ellipse_x = a * np.cos(t)
            ellipse_y = b * np.sin(t)

            # Rotate the ellipse
            R = np.array([[np.cos(theta), -np.sin(theta)],
                        [np.sin(theta), np.cos(theta)]])
            rotated_ellipse = R @ np.array([ellipse_x, ellipse_y])

            # Shift ellipse to the mean
            ellipse_x_final = rotated_ellipse[0, :] + mu[0]
            ellipse_y_final = rotated_ellipse[1, :] + mu[1]

            ax.plot(ellipse_x_final, ellipse_y_final, color, lw=.7)


        ax.scatter(x, y, color=color, label=team)
        ax.arrow(x, y, mu[0] - x, mu[1] - y, head_width = 1, head_length=.8, color='black', zorder=5)

        ax.legend()
    

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 3))

# Draw two NFL fields
for ax in axes:
    draw_nfl_field(ax)


x = [18.5, 20]
y = [11, 10]
s = [12, 4]
dir = [155/180 * np.pi, 75/180 * np.pi]
ball = [35, 25]
teams = ['offense', 'defense']
colors = ['blue', 'red']


plot_sample(x, y, s, dir, ball, teams, colors, axes[0])
axes[0].set_title('Low Influence Overlap')

x = [20, 12]
y = [10, 10]
s = [12, 4]
dir = [np.pi, 0]
ball = [25, 10]

plot_sample(x, y, s, dir, ball, teams, colors, axes[1])
axes[1].set_title('High Influence Overlap')
plt.show()


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 3))

# Draw two NFL fields
for ax in axes:
    draw_nfl_field(ax)


x = [18.5, 20]
y = [11, 10]
s = [5, 15]
dir = [155/180 * np.pi, 75/180 * np.pi]
ball = [35, 25]
teams = ['offense', 'defense']
colors = ['blue', 'red']

axes[0].scatter(x[0], y[0], color='red', label='defense')
axes[0].scatter(x[1], y[1], color='blue', label='offense')
axes[0].set_title('Low Euclidean Distance')
axes[0].legend()

